## CNN image classification (MNIST-as-JPG)

This notebook loads grayscale digit images from a directory, builds a small CNN in Keras, trains it with a `tf.data` pipeline, then runs a single-image prediction.


In [ ]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pltimg

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input


## Quick image preview

Optional sanity check: render a couple of sample images from the dataset and print the image shape.


In [ ]:
sample_img_1 = pltimg.imread("/kaggle/input/mnistasjpg/trainingSet/trainingSet/6/img_10129.jpg")
sample_img_2 = pltimg.imread("/kaggle/input/mnistasjpg/trainingSet/trainingSet/4/img_10034.jpg")

fig, axes = plt.subplots(1, 2, figsize=(6, 3))
axes[0].imshow(sample_img_1, cmap="gray")
axes[0].axis("off")
axes[1].imshow(sample_img_2, cmap="gray")
axes[1].axis("off")
plt.show()
print("sample_img_2.shape:", sample_img_2.shape)

## Data pipeline (tf.data)

Create a fast input pipeline: load images from disk, normalize to `[0, 1]`, apply augmentation on the training split, then `prefetch()` to keep the GPU busy.


In [ ]:
SEED = 42
IMG_SIZE = (28, 28)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
DATA_DIR = "/kaggle/input/mnistasjpg/trainingSet/trainingSet"
MODEL_PATH = "model2.keras"

augmentation_layer = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(40 / 360.0),
        tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        tf.keras.layers.RandomZoom(0.2),
        tf.keras.layers.RandomFlip("horizontal"),
    ]
)

rescale_layer = tf.keras.layers.Rescaling(1.0 / 255.0)

def preprocess_for_training(images, labels):
    images = rescale_layer(images)
    images = augmentation_layer(images, training=True)
    return images, labels

def preprocess_for_validation(images, labels):
    return rescale_layer(images), labels

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset="training",
).map(preprocess_for_training, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,
    seed=SEED,
    validation_split=0.2,
    subset="validation",
).map(preprocess_for_validation, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

## Model

Define a small CNN for 10-class classification, compile with Adam + categorical cross-entropy, train 20 epochs, then save in the native `.keras` format.


In [ ]:
model = Sequential(
    [
        Input(shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation="relu"),
        Conv2D(64, (3, 3), activation="relu"),
        Flatten(),
        Dense(32),
        Dense(16, activation="relu"),
        Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(train_ds, epochs=20, validation_data=val_ds)
model.save(MODEL_PATH)

## Inference example

Load the saved model and run prediction on a single image using the same normalization as training.


In [ ]:
pil_img = tf.keras.utils.load_img(
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/3/img_57.jpg",
    color_mode="grayscale",
)
img_array = tf.keras.utils.img_to_array(pil_img)
plt.imshow(img_array.squeeze(), cmap="gray")
plt.axis("off")
plt.show()

input_batch = rescale_layer(np.expand_dims(img_array, axis=0))

model = load_model(MODEL_PATH, compile=False)
pred_probs = model.predict(input_batch)
pred_class = int(np.argmax(pred_probs, axis=1)[0])
print("pred_class:", pred_class)